In [ ]:
import numpy as np
import os
import torch
from dataset import OneDDatasetLoader, OneDDataset
from network import EmbeddedNet
# from utils import reverse_normalize
import matplotlib.pyplot as plt
os.environ["CUDA_VISIBLE_DEVICES"]="1"

Load dataset, initialize model and loss criteria

In [ ]:
learning_rate = 1e-7
decay = 5e-4
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_id = range(0, 31)
eval_id = range(31, 41)
time_id = [str(i).zfill(3) for i in range(201)]
dataset = OneDDatasetLoader(
    root='/data1/tam/downloaded_datasets_bcflag1', 
    # source='/data1/tam/datasets/',
    time_id = time_id 
    # download = False
)
print(f'{len(dataset.subject_list)} subjects have been loaded.')
model = EmbeddedNet(
    input_dim_node=3, 
    input_dim_edge=6, 
    hidden_dim=128, 
    output_dim=1,  
    num_layers = 5, 
    emb=False, 
    add_self_loops=True
)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=decay)
print(f'Loaded model: {model}')
criterion = torch.nn.MSELoss()
# criterion = torch.nn.HuberLoss(delta=3.0)

Training and evaluating function

In [ ]:
def train(x, f_prev, f, p_prev, p, edge_index, edge_attr):
    model.train()
    optimizer.zero_grad()
    # print(x.size(), f_prev.size(), p_prev.size(), edge_attr.size(), edge_index.size())
    p_pred, f_pred = model(x.float(), f_prev.float(), p_prev.float(), edge_index, edge_attr.float())
    loss = criterion(f_pred, f.float())  + criterion(p_pred, p.float())
    loss.backward()
    optimizer.step()
    return loss.item()

def eval(x, f_prev, f, p_prev, p, edge_index, edge_attr):
    model.eval()
    with torch.no_grad():
        p_pred, f_pred = model(x.float(), f_prev.float(), p_prev.float(), edge_index, edge_attr.float())
        loss = criterion(f_pred, f.float()) + criterion(p_pred, p.float())
    return loss.item()

Normalize dataset

In [ ]:
def normalize(x, mean_x, std_x):
    return (x - mean_x) / (std_x + 1e-15)
def reverse_normalize(x_normed, mean_x, std_x):
    return x_normed * std_x + mean_x
def get_gen_mean_std(f, gen, max_gen=10):
    mean = []
    std = []
    for i in range(0, max_gen + 1):
        mean.append(np.mean(f[np.where(gen==i),:]))
        std.append(np.std(f[np.where(gen==i),:]))
    mean.append(np.mean(f[np.where(gen>i),:]))
    std.append(np.std(f[np.where(gen>i),:]))
    return mean, std
def normalize_gen(x, mean, std, gen, max_gen=10):
    arr =  np.array([(x[i] - mean[min(gen[i], max_gen)]) / \
        (std[min(gen[i], max_gen)] + 1e-15) for i in range(len(x))])
    return torch.tensor(arr)
def reverse_normalize_gen(x, mean, std, gen, max_gen=10):
    return np.array([x[i]*(std[min(gen[i], max_gen)] + 1e-15) + \
        mean[min(gen[i], max_gen)] for i in range(len(x))])
def normalize_dataset(dataset, device):
    x = []
    p = []
    f= []
    edge_attr = []
    # for i in range(subject, subject+1):
    for i in range(len(dataset.subject_list)):
        x.append(dataset[i].x.numpy())
        p.append(dataset[i].p.numpy())
        # f.append(np.delete(dataset[i].f.numpy(),0,0))
        f.append(dataset[i].f.numpy())
        edge_attr.append(dataset[i].edge_attr.numpy())
    x = np.concatenate(x)
    f = np.concatenate(f)
    p = np.concatenate(p)
    edge_attr = np.concatenate(edge_attr)
    gen = edge_attr[:, 2]
    mean_x = np.mean(x, axis=0)
    std_x = np.std(x, axis=0)
    mean_edge_attr = np.mean(edge_attr, axis=0)
    std_edge_attr = np.std(edge_attr, axis=0)
    mean_f = np.mean(f)
    std_f = np.std(f)
    # mean_f, std_f = get_gen_mean_std(f, np.insert(gen,0,0))
    mean_p = np.mean(p)
    std_p = np.std(p)
    # mean_p, std_p = get_gen_mean_std(p, np.insert(gen,0,0))
    data = []
    # for i in range(subject, subject+1):
    for i in range(len(dataset.subject_list)):
        data_i = dataset[i]
        gen = dataset[i].edge_attr.numpy()[:,2].astype(np.int32)
        data_i.x = normalize(data_i.x, mean_x, std_x)
        data_i.edge_attr = normalize(data_i.edge_attr, mean_edge_attr, std_edge_attr)
        data_i.p = normalize(data_i.p, mean_p, std_p)
        data_i.f = normalize(data_i.f[1:,:], mean_f, std_f)
        # data_i.p = normalize_gen(data_i.p.numpy(), mean_p, std_p, np.insert(gen,0,0))
        # data_i.f = normalize_gen(data_i.f.numpy(), mean_f, std_f, np.insert(gen,0,0))
        data.append(data_i.to(device))
    return data, mean_p, std_p, mean_f, std_f
subject = 35
# data[0].x
data, mean_p, std_p, mean_f, std_f = normalize_dataset(dataset, device)

# print(dataset.subject_list)
# train_id = range(0, 150)
# test_id = range(200, 240)

Training

In [ ]:
CUDA_LAUNCH_BLOCKING=1
torch.cuda.empty_cache()
print('Start training.')
train_loss_all = []
eval_loss_all = []
for epoch in range(100):
    train_loss = 0.0
    for i in train_id:
        x = data[i].x #.to(device)
        edge_index = data[i].edge_index #.to(device)
        edge_attr = data[i].edge_attr #.to(device)
        for time in range(1, len(time_id)):
            p_prev = data[i].p[:,time - 1].unsqueeze(1) #.to(device)
            p = data[i].p[:,time].unsqueeze(1) #.to(device)
            f_prev = data[i].f[:,time - 1].unsqueeze(1) #.to(device)
            f = data[i].f[:,time].unsqueeze(1) #.to(device)
            # print(f_prev.size(), edge_attr.size())
            train_loss += train(x, f_prev, f, p_prev, p, edge_index, edge_attr)
    train_loss /= len(train_id)
    train_loss_all.append(train_loss)
    
    eval_loss = 0.0
    for i in eval_id:
        x_eval = data[i].x
        edge_index_eval = data[i].edge_index
        edge_attr_eval = data[i].edge_attr
        for time in range(1, len(time_id)):
            p_prev_eval = data[i].p[:,time - 1].unsqueeze(1)
            p_eval = data[i].p[:,time].unsqueeze(1)
            f_prev_eval = data[i].f[:,time - 1].unsqueeze(1)
            f_eval = data[i].f[:,time].unsqueeze(1)
            eval_loss += eval(x_eval, f_prev_eval, f_eval, p_prev_eval, p_eval,\
                        edge_index_eval, edge_attr_eval)
    eval_loss /= len(eval_id)
    eval_loss_all.append(eval_loss)
    
    if epoch % 10 == 9:
        print(f'Epoch {epoch}: train loss={train_loss}; eval loss={eval_loss}')

Save model

In [ ]:
torch.save(model.state_dict(), 'models/mgn_pq_v1.pth')
plt.plot(train_loss_all, label='train_loss')
plt.plot(eval_loss_all, label='eval_loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

f = open('./predict_3/loss_mgn_pq_v1','w+')
for i in range(len(train_loss_all)):
    f.write(str(train_loss_all[i]))
    f.write('    ')
    f.write(str(eval_loss_all[i]))
    f.write('\n')
f.close()

Write prediction into 1D data files

In [ ]:
torch.cuda.empty_cache()
out_dir = './predict_3/'
model = EmbeddedNet(
    input_dim_node=3, 
    input_dim_edge=6, 
    hidden_dim=128, 
    output_dim=1,  
    num_layers = 5, 
    emb=False, 
    add_self_loops=True
)
device = torch.device("cpu")
model.load_state_dict(torch.load('models/mgn_pq_v1.pth'))
model = model.to(device)
model.eval()


def print_1D(x, edge_index, f, p, time_name, path):
    fo = open(path+'/plt_nd_000'+time_name+'.dat','w+')
    fo.write(f'VARIABLES="x" "y" "z" "p" "f"\n')
    fo.write(f'ZONE T= "plt_nd_000{time_name}.dat"\n')
    fo.write(f'N= {np.shape(x)[0]}, E= {np.shape(edge_index)[1]},F=FEPoint,ET=LINESEG\n')
    for i in range(np.shape(x)[0]):
        fo.write(f'{x[i][0]}\t{x[i][1]}\t{-1*x[i][2]}\t{p[i]}\t{f[i]}\n')
    for i in range(edge_index.shape[1]):
        fo.write(f'{edge_index[0][i] + 1}\t{edge_index[1][i] + 1}\n')
    fo.close()
import os
os.system('rm -rf {out_dir}*')

for i in range(210, 211):
    os.system('mkdir '+out_dir+dataset.subject_list[i])
    x = data[i].x.to(device)
    gen = dataset[i].edge_attr.numpy()[:,2].astype(np.int32)
    edge_index = data[i].edge_index.to(device)
    edge_attr = data[i].edge_attr.to(device)
    out_p = []
    out_f = []
    out_p.append(data[i].p[:,0].unsqueeze(1).cpu().numpy())
    out_f.append(data[i].f[:,0].unsqueeze(1).cpu().numpy())
    for time in range(1, len(time_id)):
        f_prev = data[i].f[:,time - 1].unsqueeze(1).to(device)
        p_prev = data[i].p[:,time - 1].unsqueeze(1).to(device)
        p, f = model(x.float(), f_prev.float(), p_prev.float(),edge_index, edge_attr.float())
        # p, f = model(x, f_prev, p_prev, edge_attr, edge_index)
        out_p.append(p.detach().cpu().numpy())
        out_f.append(f.detach().cpu().numpy())

    out_p = np.concatenate(out_p, axis=-1)
    out_p = reverse_normalize_gen(out_p, mean_p, std_p, np.insert(gen,0,0))

    out_f = np.concatenate(out_f, axis=-1)
    out_f = reverse_normalize_gen(out_f, mean_f, std_f, np.insert(gen,0,0))

    for j in range(len(time_id)):
        p = out_p[:,j]
        # f = np.insert(out_f[:,j],0,out_f[0,j])
        print_1D(
            x = dataset[i].x.numpy()*1e-3, 
            edge_index=dataset[i].edge_index.numpy(),
            f = f, 
            p = p, 
            time_name = time_id[j], 
            path=out_dir+dataset.subject_list[i]
        )

Visualize prediction vs groundtruth

In [ ]:
# time his
i = 35
print(dataset.subject_list[i])
path = out_dir+dataset.subject_list[i]+'/'
# path_ref = './predict_2/'+dataset.subject_list[i]+'/'
node_id = 0
time_list = [i * 4.8 / 200 for i in range(201)]
def read_value(path, node_id):
    f = open(path, 'r')
    f.readline()
    f.readline()
    f.readline()
    for i in range(node_id + 1):
        line = f.readline()
    line = line[:-1].split('\t')
    f.close()
    return float(line[-1])
value = []
value_ref = []
for time in time_id:
    value.append(read_value(path+'plt_nd_000'+time+'.dat',node_id))
    # value_ref.append(read_value(path_ref+'plt_nd_000'+time+'.dat',node_id))
y_pred = np.array(value)
# y_ref = np.array(value_ref)
y_true = dataset[i].f[node_id].numpy()
# plt.plot(time_list, y_ref, c='black', label='GCN')
plt.plot(time_list, y_pred, c='red', label='ResGCN')
plt.plot(time_list, y_true, c='blue', linestyle='dashdot', label='ground_truth')
# plt.ylim([-50,50])
plt.legend(loc='upper left')
plt.ylabel('Pressure', fontsize=20)
plt.xlabel('Time', fontsize=20)
plt.show()